In [7]:
'''import twitter
from __future__ import unicode_literals

api = twitter.Api(consumer_key=["igzzBlGJSw5wd7VnVJCQmbrIX"],
                  consumer_secret=["Bm1yD7VpvDmVSIeUOi0sEavVG5zdYtSJYdYjxUzdGZqrJTNDxc"],
                  access_token_key=["152678699-61lPSWNC22q5dkHLY3feg2V5f3zFqAL4ppvpJ3C6"],
                  access_token_secret=["qiDEzhnKj3bY4hNChH50XM3tg9aXAIuCQ8Y3Z2WIpP6zg"])


import tweepy

auth = tweepy.OAuthHandler("igzzBlGJSw5wd7VnVJCQmbrIX", "Bm1yD7VpvDmVSIeUOi0sEavVG5zdYtSJYdYjxUzdGZqrJTNDxc")
auth.set_access_token("152678699-61lPSWNC22q5dkHLY3feg2V5f3zFqAL4ppvpJ3C6", "qiDEzhnKj3bY4hNChH50XM3tg9aXAIuCQ8Y3Z2WIpP6zg")

api = tweepy.API(auth)

results = api.GetSearch(
    raw_query="q=")

search_results = api.search(q="url:http://www.usherald.com/breaking-declassified-docs-show-that-obama-admin-created-isis-in-2012-to-use-as-a-tool", count=100)

print(len(search_results))

for i in search_results:
    print(i)

for tweet in tweepy.Cursor(api.search,
                           q="google",
                           rpp=100,
                           result_type="recent",
                           include_entities=True,
                           lang="en").items():
    print tweet.created_at, tweet.text
'''


In [5]:
import sys
import os
import got3 as got


def printTweet(t):
    print("Username: %s" % t.username)
    print("Retweets: %d" % t.retweets)
    print("Text: %s" % t.text)
    print("Mentions: %s" % t.mentions)
    print("Hashtags: %s\n" % t.hashtags)
    print("id: %s\n" % t.id)
printTweet(tweet)


NameError: name 'tweet' is not defined

In [7]:
import numpy as np
import pandas as pd
from collections import Counter
import json

is_fake = []
paths = []

for root, dirs, files in os.walk("raw_source"):
    for name in files:
        if name.endswith((".json")):
            name_test = name.lower()
            if "fake" in name_test:
                is_fake+= [True]
            elif "real" in name_test:
                is_fake+= [False]
            else:
                print("Warning!!! News ", name, "not classified!")
            paths+=[root+"/"+name] 

def find(key, dictionary):
    for k, v in dictionary.items():
        if k == key:
            yield v
        elif isinstance(v, dict):
            for result in find(key, v):
                yield result
        elif isinstance(v, list):
            for d in v:
                if isinstance(d, dict):
                    for result in find(key, d):
                        yield result

final_urls = []
final_paths = []
for i in range(len(paths)):
    possible_urls = []
    json_data=open(paths[i]).read()
    data = json.loads(json_data)
    
    for k in find ("twitter", data):
        for a in find ("url", k):
            possible_urls+=[a]
    
    
    for k in find ("canonical_link", data):
        possible_urls+=[k]
    
    for k in find ("url", data):
        possible_urls+=[k]
    
    #urls cleanup
    k=0
    for j in possible_urls:
        if not isinstance(j, str):
            possible_urls.remove(j)
            continue 
        
        if not j.startswith("http"):
            possible_urls.remove(j)
            continue
            
        start_index = j.find("//")
        if start_index == -1:
            possible_urls.remove(j)
            continue
        
        new_str = j[start_index+2:]
        possible_urls[k] = new_str
        
        
        start_index = new_str.find("www.")
        if start_index != -1:
            new_str = new_str[start_index+4:]
            possible_urls[k] = new_str
        
        k+=1
    
    data = Counter(possible_urls)
    if (len(possible_urls) > 0):
        final_urls+= [max(possible_urls, key=data.get)]
        final_paths += [paths[i]]
    else:
        del is_fake[i]
    
print(len(final_urls))
print(len(is_fake))
print(len(final_paths))

383
383
383


In [123]:
def get_tweet_id_from_url(url):
    query = "url:"+url
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query).setMaxTweets(150)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    result = []
    for tweet in tweets:
        result +=[tweet.id]
    return result

In [124]:
import numpy as np
import pandas as pd

df = pd.DataFrame(columns=['source', 'fake', 'file_path', 'ids'])
print(final_urls)

a = 0
cnt = 0

final_urls = final_urls[374:]#final_urls[172:174]

for i in final_urls:
    tweets = []
    tweets+=get_tweet_id_from_url(i)
    test_url = "http://"+i
    tweets+=get_tweet_id_from_url(test_url)
    test_url = "http://www."+i
    tweets+=get_tweet_id_from_url(test_url)
    test_url = "https://"+i
    tweets+=get_tweet_id_from_url(test_url)
    test_url = "https://www."+i
    tweets+=get_tweet_id_from_url(test_url)
    
    tweets = list(set(tweets))
    if len(tweets) > 0:
        df.loc[a] = [i, is_fake[cnt], final_paths[cnt], tweets]
        a+=1
        
    cnt+=1
    print(tweets)
    


['rightwingnews.com/top-news/people-noticed-something-odd-hillarys-outfit-debate-last-night/', 'rightwingnews.com/top-news/new-york-bomb-suspect-sued-police/', 'rightwingnews.com/top-news/exposed-soros-funding-fake-veterans-pac-to-take-trump-down/', 'rightwingnews.com/barack-obama/outrage-obama-spends-770m-dollars-tax-payer-money-renovate-mosques-seas-video/', 'http://conservativebyte.com/2016/09/hillary-bad-mouths-christians/', 'occupydemocrats.com/2016/09/22/multiple-witnesses-riot-police-shoot-unarmed-charlotte-protester-head/', 'rightwingnews.com/military/boom-marines-send-terrorists-scathing-message/', 'eaglerising.com/37040/michelle-obama-attacks-hillary-if-you-cant-run-your-own-house-you-certainly-cant-run-the-white-house/', 'thepoliticalinsider.com/hillary-health-dead-brand-new-theory/', 'rightwingnews.com/donald-trump/cnn-furious-embarrassed-trump-called-them-one-major-thing/', 'thepoliticalinsider.com/first-presidential-debate-hillary-clinton-sick/', 'occupydemocrats.com/2016

['777701206093402112', '777700927641747456', '777827495504703488', '777742267893190656', '777692263019343872', '777721522144280576', '777703425266094080', '777810161121492993', '777897766416748544', '777706100380409856', '777834629009174528', '777869893974814720', '777845002613424128', '777823274583592960', '777839031837220864', '779133688483188736', '777729692539375616', '777692325476720640', '777692306157830144', '777710609433591808', '777878666432962561', '777881067592220672', '777693316531953665', '777942513126486017', '777825259735003136', '777824904641122304', '778294050952613888', '777726871572066306', '777839660651450368', '777882193557270528', '777692066495070208', '777810228100333568', '777700673311899649', '777911133130850305', '777847758258511872', '777869910298984449', '777753229157212160', '777943675795939329', '777713926607953920', '777707977671180288', '777867877290573824', '777869910886187009', '777705809505640448', '777921338220515329', '778218646510379008', '77830700

['778111809232535552', '778032219013074944', '777896227476082693', '777895533448794112', '777900153298571264', '777896649981124608', '777896123134423046', '777938202199982080', '777896114557116421', '777950977315876864', '777896443281412103', '777896247717793793', '777942053888069632', '777897293366243329', '778006043737464832', '777988907409346560', '778049373116723200', '777896409760538624', '777910863541964800', '777910518254149633', '777896107959476224', '777898928352079872', '777900343178821632', '778025271773556736', '777896461539287040', '777902278195970048', '777989027333042176', '777987595414474752', '777911733344165888', '777910534137978880', '778035972164481024', '777903232542736385', '777964957329715201', '777899554121388032', '777900846982004736', '777900720792211456', '777987257013825538', '777918428980056064', '777895520391999488', '777903290432516097', '777919131039494144', '777897306209169409', '777910531122343939', '777896114871603200', '777992323368161284', '77792060

In [125]:
df

,source,fake,file_path,ids
0,http://conservativebyte.com/2016/09/inside-the...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[777701206093402112, 777700927641747456, 77782..."
1,cnn.com/2016/09/19/politics/syria-coalition-ai...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[777979404152557569, 777979190586912768, 77797..."
2,cnn.com/2016/09/20/politics/terence-crutcher-t...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[778643210503655428, 778671291775582208, 77866..."
3,cnn.com/2016/09/19/politics/obama-to-us-dont-s...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[778111809232535552, 778032219013074944, 77789..."
4,cnn.com/2016/09/21/politics/hillary-clinton-po...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[778702245646864384, 778699231997468674, 77869..."
5,ifyouonlynews.com/humor/emmy-host-kimmel-calls...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[778073702080647168, 777886367678205952, 77808..."
6,oppositionreport.com/donald-trump-jr-just-pinn...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,[871002767313309696]
7,cnn.com/2016/09/19/politics/terry-mcauliffe-wi...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[778607444398370817, 777922362930925572, 77792..."
8,politico.com/story/2016/09/trump-foundation-wa...,True,raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_...,"[779092930598502400, 778574479639257089, 77857..."


In [126]:
df.to_csv("partial_dataset_6.txt", sep=";", index=False)

In [109]:
cnt

61

In [11]:
df1 = pd.read_csv("partial_dataset_1.txt", sep=";") 
df2 = pd.read_csv("partial_dataset_2.txt", sep=";") 
df3 = pd.read_csv("partial_dataset_3.txt", sep=";") 
df4 = pd.read_csv("partial_dataset_4.txt", sep=";") 
df5 = pd.read_csv("partial_dataset_5.txt", sep=";")
final_df = pd.concat([df1, df2, df3, df4, df5])
final_df = final_df.reset_index(drop=True)

In [13]:
def assign_path(str_url):
    ind = final_urls.index(str_url)
    return final_paths[ind]

def assign_label(str_url):
    ind = final_urls.index(str_url)
    return is_fake[ind]
    
final_df["file_path"] = final_df.apply(lambda row: assign_path(row['source']), axis=1)
final_df["fake"] = final_df.apply(lambda row: assign_label(row['source']), axis=1)

In [14]:
pd.set_option('max_colwidth',80)
final_df[final_df["file_path"].duplicated()]["file_path"]

21          raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_7-Webpage.json
53         raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_11-Webpage.json
60         raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_13-Webpage.json
96     raw_source/politi_RealNewsContent/PolitiFact_Real_26-Webpage.json
114     raw_source/politi_RealNewsContent/PolitiFact_Real_8-Webpage.json
118        raw_source/buzz_FakeNewsContent/BuzzFeed_Fake_10-Webpage.json
125    raw_source/politi_RealNewsContent/PolitiFact_Real_42-Webpage.json
Name: file_path, dtype: object

In [15]:
final_df.drop_duplicates(inplace=True)
final_df = final_df.reset_index(drop=True)

In [18]:
final_df.to_csv("final_dataset.txt", sep=";", index=False)

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(final_df["source"])

0      rightwingnews.com/barack-obama/outrage-obama-spends-770m-dollars-tax-payer-m...
1                   http://conservativebyte.com/2016/09/hillary-bad-mouths-christians/
2      occupydemocrats.com/2016/09/22/multiple-witnesses-riot-police-shoot-unarmed-...
3            rightwingnews.com/military/boom-marines-send-terrorists-scathing-message/
4      eaglerising.com/37040/michelle-obama-attacks-hillary-if-you-cant-run-your-ow...
5                        thepoliticalinsider.com/hillary-health-dead-brand-new-theory/
6      rightwingnews.com/donald-trump/cnn-furious-embarrassed-trump-called-them-one...
7              thepoliticalinsider.com/first-presidential-debate-hillary-clinton-sick/
8      occupydemocrats.com/2016/09/23/bombshell-abc-reveals-trump-paid-hundreds-mil...
9      usherald.com/breaking-obama-just-disgrace-u-s-surrendering-isis-front-entire...
10     proudcons.com/clinton-foundation-caught-in-their-most-disturbing-scheme-yet-...
11                 freedomdaily.com/charlot